In [1]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

def getPageSize(path):
    pdf = PdfFileReader(open(path, 'rb'))
    page_1 = pdf.getPage(0)
    if page_1.get('/Rotate', 0) in [90, 270]:
        return page_1['/MediaBox'][2], page_1['/MediaBox'][3]
    else:
        return page_1['/MediaBox'][3], page_1['/MediaBox'][2]
    

path = 'lec.pdf'
h,w=getPageSize(path)
print(h,w)

packet = io.BytesIO()
can = canvas.Canvas(packet, pagesize=  (w, h +100)   )
can.drawString(75,85, "Notes:")
can.drawString(75,65, "Yusuf Ammar")
can.drawString(75,50, "Yusuf Ammar2")
can.drawString(75,35, "Yusuf Ammar2")
can.drawString(75,20, "Yusuf Ammar2")
can.drawString(75, 5, "Yusuf Ammar2")
can.save()
#move to the beginning of the StringIO buffer
packet.seek(0)


# create a new PDF with Reportlab
new_pdf = PdfFileReader(packet)

# read your existing PDF
existing_pdf = PdfFileReader(open("lec.pdf", "rb"))
output = PdfFileWriter()

page = new_pdf.getPage(0)
page.mergeScaledTranslatedPage( existing_pdf.getPage(0) , 1 , 0 , 100) 


#page= new_pdf.getPage(0)
output.addPage(page)
# finally, write "output" to a real file
outputStream = open("destination.pdf", "wb")
output.write(outputStream)
outputStream.close()

path = 'destination.pdf'
h,w=getPageSize(path)
print(h,w)

540 960
640 960


In [2]:
#Match Notes to pdf file (with iteration)

from PyPDF2 import PdfFileWriter, PdfFileReader
import io
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont 

import arabic_reshaper
from bidi.algorithm import get_display



def getPageSize(path, pageNo):
    pdf = PdfFileReader(open(path, 'rb'))
    page_1 = pdf.getPage(pageNo)
    if page_1.get('/Rotate', 0) in [90, 270]:
        return page_1['/MediaBox'][2], page_1['/MediaBox'][3]
    else:
        return page_1['/MediaBox'][3], page_1['/MediaBox'][2]
    
    
def outputPDF(path, strings):          #Input: path -> pdf file path, strings -> list of strings (first entry is the notes of 1st slide directly no slide 0) | Output: new pdf file with notes added (destination.pdf)
    
    originalPDF = PdfFileReader(open(path, "rb"))
    totalpages = originalPDF.numPages

    output = PdfFileWriter()

    
    #inside loop
    for j in range(totalpages): #iterate over each slide
                
        slideNotes= strings[j] #1 string       #notes of corrosponding slide
        
        #split slide notes on 100 charchters (on spaces)
        
        
        stringsList=[]
        counter=0
        currentString=""

        for i in range(len(slideNotes)): 
            currentCharacter = slideNotes[i]

            if counter<120:
                currentString+= currentCharacter
                counter+=1
            else:
                if currentCharacter!= " ":
                   currentString+= currentCharacter
                   counter+=1
                else:                                  #stopiing condition 1
                    currentString+= currentCharacter
                    stringsList.append(currentString)
                    counter=0
                    currentString= ""

            if i==len(slideNotes)-1:   #stoppiing condition 2
                stringsList.append(currentString)
                    
            
        #now string of slide is split into 100 character strings in strings list array
                       
        h,w=getPageSize(path,j)
       
                
        #Create New Page
        addedHeight= 45 + (15* len(stringsList) )
        packet = io.BytesIO()
        can = canvas.Canvas(packet, pagesize=  (w, h + addedHeight)   )           # page size= (w,h)
        
        #can.setStrokeColorRGB(0,0,1)
        can.rect(65,5,w-130,addedHeight-10, fill=0)
        
        currentHeight= addedHeight-20
        can.setFillColorRGB(0,0,1)
        
       
        pdfmetrics.registerFont(TTFont('arial', 'arial.ttf'))
        can.setFont('arial', 12)
        can.drawString(75, currentHeight, "NOTES")
        currentHeight-=25
        
        can.setFillColorRGB(0,0,0)
        for x in stringsList:
            x= "I      " + x
            ar = arabic_reshaper.reshape(x)
            ar = get_display(ar)
            can.drawString(75, currentHeight,  ar)
            currentHeight-=15
            
        can.save()
        packet.seek(0)
        new_pdf = PdfFileReader(packet)

        page = new_pdf.getPage(0)
        page.mergeScaledTranslatedPage( originalPDF.getPage(j) , 1 , 0 , addedHeight)    #merge old page with new page (so the page would have the size of new page)

        output.addPage(page)
   
    
    #out of loop
    outputStream = open("destination.pdf", "wb")
    output.write(outputStream)
    outputStream.close()



In [3]:
list1=["اللغة العربية رائعة The Arabic language  اللغة العربية رائعة is a morphologically rich and اللغة العربية رائعة complex اللغة العربية رائعة language with relatively little resources and a less explored syntax compared to English. Given these limitations, tasks like Sentiment Analysis (SA), Named Entity Recognition (NER), and Question Answering (QA), have proven to be very challenging to tackle. Recently, with the surge of transformers based models, language-specific BERT based models proved to have a very efficient understanding of languages, provided they are pre-trained on a very large corpus. Such models were able to set new standards and achieve state-of-the-art results for most NLP tasks. In this paper, we pre-trained BERT specifically for the Arabic language in the pursuit of achieving the same success that BERT did for the English language. We then compare the performance of AraBERT with multilingual BERT provided by Google and other state-of-the-art approaches. The results of the conducted experiments show that the newly developed AraBERT achieved state-of-the-art results on most tested tasks. The pretrained araBERT models are publicly available on github.com/aub-mind/araBERT hoping to encourage research and applications for Arabic NLP.", "sdffds", "dvdfvdvds"]


path="Sample.pdf"
outputPDF(path,list1)